# Tannic_Sentiments

#### Based on https://colab.research.google.com/drive/1OlQpHdZD7zVyZW56r8vI-L8BYylq_Umm#scrollTo=XhAOS8BY0jC2

## A Sentiment Classifier that has been triained on wine reviews. 

In [13]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Softmax, Embedding, GlobalMaxPool1D, GlobalAvgPool1D, Concatenate, Bidirectional, SpatialDropout1D

In [14]:
wine_df = pd.read_csv('ranked_wine.csv')
wine_df.sample(3)

,Unnamed: 0,description,points,sentiment
89956,67432,"89-91 Barrel sample. Caramel aromas, a wine th...",90,1
31974,78476,Fresh aromas of red fruits are immediately att...,86,0
127496,117214,The longer this wine breathes in the glass the...,94,1


In [15]:
# trainable_df = wine_df[['description', 'points']]
# trainable_df.head()

In [16]:
tokenizer = Tokenizer()

In [17]:
tokenizer.fit_on_texts(wine_df.description)

In [18]:
tokenizer.texts_to_sequences(['I need a test string, and this will do.'])

[[2340, 606, 3, 6940, 4521, 1, 6, 87, 834]]

In [19]:
pad_sequences(tokenizer.texts_to_sequences(["This is awful.", "I just want my code to work.", "and to not have excessive and silly bloat in it."]), 50)

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     6,     7, 14546],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,  2340,   106,
         1048,  5066, 11727,    11,  1076],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,    

In [20]:
x = pad_sequences(tokenizer.texts_to_sequences(wine_df), 50)
y = wine_df.points

In [21]:
input = Input((50,), name='input')
embed = Embedding(len(tokenizer.word_index), 100)(input)
embed_dropout = SpatialDropout1D(0.5)(embed)
rnn = Bidirectional(GRU(50,return_sequences=True, recurrent_dropout=0.2))(embed_dropout)
maxed = GlobalMaxPool1D()(rnn)
avged = GlobalAvgPool1D()(rnn)
concat = Concatenate()([maxed, avged])
dense = Dense(3, activation='softmax')(concat)

model = Model(input, dense)

In [22]:
model.compile('adam', 'sparse_categorical_crossentropy', ['sparse_categorical_accuracy'])

In [23]:
model.fit(x, y, batch_size=512, validation_split=0.2, epochs=3)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>

In [0]:
test = model.predict(pad_sequences(tokenizer.texts_to_sequences(["Hello", 'This is the worst', 'This has depth', 'There is feeling'])))
test